In [2]:
#import os
#import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline
#from PIL import Image

# NOT SURE WHAT THE POINT OF THE CODE BELOW IS

#if torch.cuda.is_available():
#    torch.backends.cuda.deterministic = True

In [ ]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
#NUM_FEATURES = 32*32
NUM_CLASSES = 2 # not sure this is needed

# Other
if torch.cuda.is_available():
    DEVICE = "cuda:0"
else:
    DEVICE = "cpu"


In [ ]:
# read data
df_train = pd.read_csv('log_train.csv')
df_test = pd.read_csv('log_test.csv')

# get data matrix and labels
train_data = df_train.iloc[:, 1:]
train_labels = df_train.iloc[:, 0]
test_data = df_test.iloc[:, 1:]
test_labels = df_test.iloc[:, 0]

data_dims = (19, 44)

In [ ]:
class logDataset(Dataset):
    """Log Anomaly Features Dataset"""
    
    def __init__(self, data_vec, dims, labels=None, transforms=None):
        self.X = data_vec
        self.dims = dims
        self.y = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        data_matrix = self.X.iloc[idx, :]
        data_matrix = np.asarray(data_matrix).astype(np.float64).reshape(dims, 1)

        if self.transforms:
            data_matrix = self.transforms(data_matrix)
        
        if self.y:
            return(data_matrix, self.y[i])
        else:
            return data_matrix


In [ ]:
# NOTE THAT AS WE'VE DONE TF-IDF, WE MAY NOT WANT TO NORMALIZE
# HOWEVER, ONE OF THE ODD THINGS HERE IS THAT SINCE WE'RE PASSING FILTERS OVER THINGS
# IF THE COLUMNS REPRESENT SLIGHTLY DIFFERENT THINGS IT MAY BE AN ISSUE, ESPECIALLY FOR POOLING?

# define transforms
#train_mean = (0.5, 0.5, 0.5)
#train_std = (0.5, 0.5, 0.5)

apply_transforms = transforms.Compose([#transforms.Resize((32, 32)),
                                       transforms.ToTensor(),
                                       #transforms.Normalize(train_mean, train_std)])


In [ ]:
train_dataset = logDataset(train_data, dims = data_dims, transforms = apply_transforms)
test_dataset = logDataset(test_data, dims = data_dims, transforms = apply_transforms)

In [ ]:
# use DataLoader

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=8,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

# Checking the dataset
for data, labels in train_loader:  
    print('Image batch dimensions:', data.shape)
    print('Image label dimensions:', data.shape)
    break

# Checking the dataset
for data, labels in train_loader:  
    print('Image batch dimensions:', data.shape)
    print('Image label dimensions:', data.shape)
    break

In [ ]:
# this is something to do with warming up the nn
# NEED TO FIGURE THIS OUT AND KEEP OR REMOVE
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

In [ ]:
##########################
### MODEL
##########################


class logCNN(nn.Module):

    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        self.features = nn.Sequential(
            
            nn.Conv2d(1, 16, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 32*in_channels, kernel_size=2),
            nn.ReLU(), # should there be a ReLU here or not? there wasn't after the 2D pooling layer
            #nn.MaxPool2d(kernel_size=3)
        )

        self.classifier = nn.Sequential(
            nn.Linear(32*4*4, 120*in_channels),
            nn.ReLU(),
            nn.Linear(120*in_channels, 84*in_channels),
            nn.ReLU(),
            nn.Linear(84*in_channels, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [ ]:
torch.manual_seed(RANDOM_SEED)

model = logCNN(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)